In [1]:
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM
from alch_deriv import alch_deriv
from pyscf import gto,scf
import numpy as np 
from numpy.linalg import norm as norm
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
import basis_set_exchange as bse
from FDcoeffs import *
from pyscf.geomopt.berny_solver import optimize
ang2bohr=1.8897261246
bohr2ang=.5291772109

In [3]:
l=3.3376
x=-(1/24)**.5*l
z=(2/3)**.5*l+x
ch4str="C 0 0 0; H 0 0  {}; H  0  {} {}; H {} {} {}; H {} {} {}".format(z,l/3**.5,x,l/2,-l*3**.5/6,x,-l/2,-l*3**.5/6,x)

In [4]:
CH4=gto.M(atom=ch4str,unit="bohr",basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
mf = scf.RHF(CH4)
mf.scf()

converged SCF energy = -40.2151804034984


-40.215180403498415

In [5]:
g0=mf.Gradients().grad()

--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000    -0.0000000000    -0.0000000000
1 H     0.0000000000    -0.0000000000    -0.0000007926
2 H     0.0000000000    -0.0000007473     0.0000002642
3 H    -0.0000006472     0.0000003736     0.0000002642
4 H     0.0000006472     0.0000003736     0.0000002642
----------------------------------------------


In [6]:
h0=mf.Hessian().hess()

In [7]:
ch4coords=CH4.atom_coords()

In [8]:
afC=aaff(mf,[1,0,0,0,0])+alc_deriv_grad_nuc(CH4,[1,0,0,0,0])
afH1=aaff(mf,[0,0,0,0,-1])+alc_deriv_grad_nuc(CH4,[0,0,0,0,-1])
afH2=aaff(mf,[0,0,0,-1,0])+alc_deriv_grad_nuc(CH4,[0,0,0,-1,0])
afH3=aaff(mf,[0,0,-1,0,0])+alc_deriv_grad_nuc(CH4,[0,0,-1,0,0])

In [9]:
alc_deriv_grad_nuc(CH4,[0,0,-1,0,0])

array([[ 0.        , -1.35417611,  0.47877356],
       [ 0.        , -0.05182876,  0.07329693],
       [ 0.        ,  1.56149114, -0.55207049],
       [ 0.04488502, -0.07774314,  0.        ],
       [-0.04488502, -0.07774314,  0.        ]])

In [10]:
for i in (afH1+afC):
    print(norm(i))

0.031128756363954634
0.06943666146069344
0.06943666146068825
0.06943666146072108
0.004045775548047937


In [11]:
for i in range(5):
    print(norm(h0[i,i]))

1.034568584181829
0.375974893176426
0.3759748931765666
0.3759748931765299
0.37597489317655686


In [12]:
.37597489317647576*.01815090704294456

0.006824285336527222

In [13]:
afH1

array([[-0.02541652, -0.01467424, -0.01037625],
       [-0.01250345, -0.00721887, -0.01100032],
       [-0.01250345, -0.01277749, -0.00313924],
       [-0.01731736, -0.00443956, -0.00313924],
       [ 0.06774078,  0.03911016,  0.02765506]])

In [14]:
for i in afH1:
    print(norm(i))

0.031128756363946408
0.018150907042911068
0.018150907042930556
0.018150907042960622
0.08296517664323332


In [15]:
nh3str="""N  -0.000000  -0.128885  -0.043079;   H   0.000000   0.067780   0.935168;   
   H   0.808204   0.295141  -0.446045;  
   H  -0.808204   0.295141  -0.446045;
   """

In [16]:
#NH3=gto.M(atom="N 0 0 0;H 0 0 1.8; H 0.9 0.5 -.9; H -0.9 0.5 -.9",unit="bohr",
NH3=gto.M(atom=nh3str,
            basis={"H":"pc-2",'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])})
mfnh3 = scf.RHF(NH3)
mfnh3.scf()

converged SCF energy = -56.2221835947892


-56.22218359478923

In [17]:
nh3_eq=optimize(mfnh3)


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   N   0.000000  -0.128885  -0.043079    0.000000  0.000000  0.000000
   H   0.000000   0.067780   0.935168    0.000000  0.000000  0.000000
   H   0.808204   0.295141  -0.446045    0.000000  0.000000  0.000000
   H  -0.808204   0.295141  -0.446045    0.000000  0.000000  0.000000
converged SCF energy = -56.2221835947894
--------------- SCF_Scanner gradients ---------------
         x                y                z
0 N     0.0000000000     0.0000180200    -0.0001418131
1 H     0.0000000000     0.0000355141     0.0000167946
2 H    -0.0000802407    -0.0000267671     0.0000625092
3 H     0.0000802407    -0.0000267671     0.0000625092
----------------------------------------------
cycle 1: E = -56.2221835948  dE = -56.2222  norm(grad) = 0.000210009


In [18]:
nh3optg=nh3_eq.atom_coords()

In [19]:
NH3bs=gto.M(atom=nh3str,
            basis={"H":"pc-2",'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
mfnh3bs = scf.RHF(NH3bs)
mfnh3bs.scf()

converged SCF energy = -56.2217786977168


-56.221778697716815

In [20]:
nh3_eqbs=optimize(mfnh3)


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   N   0.000000  -0.128885  -0.043079    0.000000  0.000000  0.000000
   H   0.000000   0.067780   0.935168    0.000000  0.000000  0.000000
   H   0.808204   0.295141  -0.446045    0.000000  0.000000  0.000000
   H  -0.808204   0.295141  -0.446045    0.000000  0.000000  0.000000
converged SCF energy = -56.2221835947892
--------------- SCF_Scanner gradients ---------------
         x                y                z
0 N    -0.0000000000     0.0000180215    -0.0001418129
1 H     0.0000000000     0.0000355138     0.0000167934
2 H    -0.0000802417    -0.0000267677     0.0000625097
3 H     0.0000802417    -0.0000267677     0.0000625097
----------------------------------------------
cycle 1: E = -56.2221835948  dE = -56.2222  norm(grad) = 0.00021001


In [21]:
norm(nh3optg[1]-nh3optg[0])

1.8856061076490165

In [22]:
OH2=gto.M(atom="O 0 0 0;H 0 -0.15229 1.76907; H 0   1.7401 -0.35319",unit="bohr",
            basis={"H":"pc-2",'O':bse.get_basis("pcX-2",fmt="nwchem",elements=[8])})
mfoh2 = scf.RHF(OH2)
mfoh2.scf()

converged SCF energy = -76.0633193329877


-76.06331933298767

In [23]:
oh2_eq=optimize(mfoh2)


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   H   0.000000  -0.080588   0.936152    0.000000  0.000000  0.000000
   H   0.000000   0.920821  -0.186900    0.000000  0.000000  0.000000
converged SCF energy = -76.063319332988
--------------- SCF_Scanner gradients ---------------
         x                y                z
0 O    -0.0000000000     0.0000002303    -0.0000207451
1 H     0.0000000000     0.0000064495     0.0000117997
2 H     0.0000000000    -0.0000066798     0.0000089454
----------------------------------------------
cycle 1: E = -76.063319333  dE = -76.0633  norm(grad) = 2.71272e-05


In [24]:
oh2optg=oh2_eq.atom_coords()

In [25]:
oh2optg

array([[ 0.     ,  0.     ,  0.     ],
       [ 0.     , -0.15229,  1.76907],
       [ 0.     ,  1.7401 , -0.35319]])

In [26]:
FH=gto.M(atom="F 0 0 0;H 0 0 1.695741",unit="bohr",
            basis={"H":"pc-2",'F':bse.get_basis("pcX-2",fmt="nwchem",elements=[9])})
mffh = scf.RHF(FH)
mffh.scf()
fh_eq=optimize(mffh)

converged SCF energy = -100.065540168732

Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   F   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   H   0.000000   0.000000   0.897347    0.000000  0.000000  0.000000
converged SCF energy = -100.065540168733
--------------- SCF_Scanner gradients ---------------
         x                y                z
0 F    -0.0000000000     0.0000000000    -0.0000009944
1 H     0.0000000000    -0.0000000000     0.0000009944
----------------------------------------------
cycle 1: E = -100.065540169  dE = -100.066  norm(grad) = 1.40635e-06


In [27]:
fhoptg=fh_eq.atom_coords()

In [28]:
#ch4coords,h0,afC,afH1,nh3optg,oh2optg,fhoptg

In [29]:
#getting inverse of the hessian matrix
H1=np.linalg.inv((np.swapaxes(h0,1,2)).reshape(15,15))
h1=H1.reshape(5,3,5,3)
def invhess(h):
    H=np.linalg.inv((np.swapaxes(h,1,2)).reshape(h.shape[0]*h.shape[2],h.shape[0]*h.shape[2]))
    h1=H.reshape(h.shape[0],h.shape[2],h.shape[1],h.shape[3])
    return h1

In [30]:
nh3predictg=(ch4coords-np.einsum('ijkl,kl->ij',h1,  afC+afH1))[:4]
oh2predictg=(ch4coords-np.einsum('ijkl,kl->ij',h1,  2*afC+afH1+afH2))[:3]
fhpredictg=(ch4coords- np.einsum('ijkl,kl->ij',h1,  3*afC+afH1+afH2+afH3))[:2]

In [31]:
#place non H atom in the origin
nh3predictg-=nh3predictg[0]
oh2predictg-=oh2predictg[0]
fhpredictg-=fhpredictg[0]
nh3optg-=nh3optg[0]
oh2optg-=oh2optg[0]
fhoptg-=fhoptg[0]

In [32]:
#bond lengths
print(norm(ch4coords[1]))
print(norm(nh3predictg[1]),norm(nh3optg[1]))
print(norm(oh2predictg[1]),norm(oh2optg[1]))
print(norm(fhpredictg[1]),norm(fhoptg[1]))

2.043854241378284
1.8771027192181162 1.8856061076490165
1.7017431491183759 1.7756128263222248
1.5148049054776223 1.695741


In [33]:
def anglecenter(coords):
    return np.around(np.arccos(coords[1].dot(coords[2])/norm(coords[1])/norm(coords[2]))*180/np.pi,decimals=3)

In [34]:
#angles  
print(anglecenter(ch4coords))
print(anglecenter(nh3predictg),anglecenter(nh3optg))
print(anglecenter(oh2predictg),anglecenter(oh2optg))

109.471
103.102 108.193
96.677 106.394


In [35]:
oh2optg

array([[ 0.     ,  0.     ,  0.     ],
       [ 0.     , -0.15229,  1.76907],
       [ 0.     ,  1.7401 , -0.35319]])

In [36]:
"""ipv.figure()
ipv.scatter(ch4coords[:,0],ch4coords[:,1],ch4coords[:,2],color="black",marker='sphere',size=12)
ipv.scatter(nh3predictg[:,0],nh3predictg[:,1],nh3predictg[:,2],color="blue",marker='sphere',size=12)
ipv.scatter(oh2predictg[:,0],oh2predictg[:,1],oh2predictg[:,2],color="green",marker='sphere',size=12)
ipv.scatter(fhpredictg[:,0],fhpredictg[:,1],fhpredictg[:,2],color="yellow",marker='sphere',size=12)
ipv.scatter(np.asarray([0.]),np.asarray([0.]),np.asarray([0.]),color="red",marker='sphere',size=12)
ipv.show()"""
pass

In [37]:
#ipv.figure()
#ipv.scatter(ch4coords[:,0],ch4coords[:,1],ch4coords[:,2],color="black",marker='sphere',size=12)
#ipv.scatter(nh3optg[:,0],nh3optg[:,1],nh3optg[:,2],color="blue",marker='sphere',size=12)
#ipv.scatter(oh2optg[:,0],oh2optg[:,1],oh2optg[:,2],color="green",marker='sphere',size=12)
#ipv.scatter(fhoptg[:,0],fhoptg[:,1],fhoptg[:,2],color="yellow",marker='sphere',size=12)
#ipv.show()

In [38]:
np.arccos(ch4coords[1].dot(ch4coords[2])/norm(ch4coords[1])/norm(ch4coords[2]))

1.9106332362490186

## Rotations 

In [39]:
rv=np.asarray([0,0,1])*np.pi*2/3
r = R.from_rotvec(rv)

In [40]:
np.set_printoptions(precision=5,suppress=True)

In [41]:
ch4coords,r.apply(ch4coords),r.apply(r.apply(ch4coords)),r.apply(r.apply(r.apply(ch4coords)))

(array([[ 0.     ,  0.     ,  0.     ],
        [ 0.     ,  0.     ,  2.04385],
        [ 0.     ,  1.92696, -0.68128],
        [ 1.6688 , -0.96348, -0.68128],
        [-1.6688 , -0.96348, -0.68128]]),
 array([[ 0.     ,  0.     ,  0.     ],
        [ 0.     ,  0.     ,  2.04385],
        [-1.6688 , -0.96348, -0.68128],
        [ 0.     ,  1.92696, -0.68128],
        [ 1.6688 , -0.96348, -0.68128]]),
 array([[ 0.     ,  0.     ,  0.     ],
        [ 0.     ,  0.     ,  2.04385],
        [ 1.6688 , -0.96348, -0.68128],
        [-1.6688 , -0.96348, -0.68128],
        [ 0.     ,  1.92696, -0.68128]]),
 array([[ 0.     ,  0.     ,  0.     ],
        [ 0.     ,  0.     ,  2.04385],
        [ 0.     ,  1.92696, -0.68128],
        [ 1.6688 , -0.96348, -0.68128],
        [-1.6688 , -0.96348, -0.68128]]))

In [42]:
ch4coords

array([[ 0.     ,  0.     ,  0.     ],
       [ 0.     ,  0.     ,  2.04385],
       [ 0.     ,  1.92696, -0.68128],
       [ 1.6688 , -0.96348, -0.68128],
       [-1.6688 , -0.96348, -0.68128]])

In [43]:
afH1

array([[-0.02542, -0.01467, -0.01038],
       [-0.0125 , -0.00722, -0.011  ],
       [-0.0125 , -0.01278, -0.00314],
       [-0.01732, -0.00444, -0.00314],
       [ 0.06774,  0.03911,  0.02766]])

In [44]:
afH2

array([[ 0.02542, -0.01467, -0.01038],
       [ 0.0125 , -0.00722, -0.011  ],
       [ 0.0125 , -0.01278, -0.00314],
       [-0.06774,  0.03911,  0.02766],
       [ 0.01732, -0.00444, -0.00314]])

In [45]:
r.apply(afH1)

array([[ 0.02542, -0.01467, -0.01038],
       [ 0.0125 , -0.00722, -0.011  ],
       [ 0.01732, -0.00444, -0.00314],
       [ 0.0125 , -0.01278, -0.00314],
       [-0.06774,  0.03911,  0.02766]])

In [46]:
b=r.apply(afH1)
b[[2,3,4]]=b[[3,4,2]]
b #=afH2

array([[ 0.02542, -0.01467, -0.01038],
       [ 0.0125 , -0.00722, -0.011  ],
       [ 0.0125 , -0.01278, -0.00314],
       [-0.06774,  0.03911,  0.02766],
       [ 0.01732, -0.00444, -0.00314]])

In [47]:
afH3

array([[-0.     ,  0.02935, -0.01038],
       [-0.     ,  0.01444, -0.011  ],
       [ 0.     , -0.07822,  0.02766],
       [-0.00481,  0.01722, -0.00314],
       [ 0.00481,  0.01722, -0.00314]])

In [48]:
c=r.apply(r.apply(afH1))

In [49]:
#r.apply(r.apply(r.apply(afH1)))

## First derivative 
$$ APDFT1(g)=  \sum_I\frac{\partial g}{\partial \lambda_I} =\sum_I F_I $$
all derivative are equal and can be computed analytically , $F$ is the alchemical force. 
## Second derivative 
$$ APDFT2(g)= \sum_I (\frac{\partial F_I}{\partial \lambda_I} 
+2 \frac{\partial }{\partial \lambda_I}\sum_{J\ne I} F_J )
$$

## Third derivative 
$$ APDFT3(g)= \sum_K \sum_I (\frac{\partial F_I}{\partial \lambda_I} 
+2 \frac{\partial }{\partial \lambda_I}\sum_{J\ne I} F_J )
$$ 

In [50]:
#p , m , pp, mm
dl=.05

In [51]:
CH4_p=FcM(fcs=[dl,0,0,0,-dl],atom=ch4str,unit="bohr",basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
mf_p = scf.RHF(CH4_p)
mf_p.scf(dm0=mf_p.init_guess_by_1e())

converged SCF energy = -40.9027706457901


-40.90277064579013

In [52]:
CH4_m=FcM(fcs=[-dl,0,0,0,dl],atom=ch4str,unit="bohr",basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
mf_m = scf.RHF(CH4_m)
mf_m.scf(dm0=mf_m.init_guess_by_1e())

converged SCF energy = -39.5399509316286


-39.53995093162855

In [53]:
af_p1=aaff(mf_p,[1,0,0,0,-1])+alc_deriv_grad_nuc(CH4_p,[1,0,0,0,-1])
af_m1=aaff(mf_m,[1,0,0,0,-1])+alc_deriv_grad_nuc(CH4_m,[1,0,0,0,-1])
af_1=afH1+afC

In [54]:
(alc_deriv_grad_nuc(CH4_p,[1,0,0,0,-1])-alc_deriv_grad_nuc(CH4_m,[1,0,0,0,-1]))/(2*dl)/2,\
alc_differential_grad_nuc(CH4,[1,0,0,0,-1])-alc_deriv_grad_nuc(CH4,[1,0,0,0,-1])

(array([[ 0.19546,  0.11285,  0.0798 ],
        [ 0.     ,  0.     ,  0.     ],
        [ 0.     ,  0.     ,  0.     ],
        [ 0.     ,  0.     ,  0.     ],
        [-0.19546, -0.11285, -0.0798 ]]),
 array([[ 0.19546,  0.11285,  0.0798 ],
        [ 0.     ,  0.     ,  0.     ],
        [ 0.     ,  0.     ,  0.     ],
        [ 0.     ,  0.     ,  0.     ],
        [-0.19546, -0.11285, -0.0798 ]]))

In [55]:
g_nh3_2=af_1+(af_p1-af_m1)/(2*dl)/2
#g_nh3_2=aaff(mf,[1,0,0,0,-1])+(aaff(mf_p,[1,0,0,0,-1])-aaff(mf_m,[1,0,0,0,-1]))/.8+alc_differential_grad_nuc(CH4,[1,0,0,0,-1])
g_nh3_3=g_nh3_2+(af_p1-2*af_1+af_m1)/dl**2/6

In [56]:
nh3predictg=(ch4coords-np.einsum('ijkl,kl->ij',h1,  afC+afH1))[:4]
nh3predictg2=(ch4coords-np.einsum('ijkl,kl->ij',h1, g_nh3_2))[:4]
nh3predictg3=(ch4coords-np.einsum('ijkl,kl->ij',h1, g_nh3_3))[:4]
nh3predictg-=nh3predictg[0]
nh3predictg2-=nh3predictg2[0]
nh3predictg3-=nh3predictg3[0]

In [57]:
norm(nh3optg[3]),norm(nh3predictg[3]),norm(nh3predictg2[3]),norm(nh3predictg3[3])

(1.8853495071094928,
 1.8771041371463273,
 1.8673665951584084,
 1.8742428039116006)

In [58]:
anglecenter(nh3optg),anglecenter(nh3predictg),anglecenter(nh3predictg2),anglecenter(nh3predictg3)

(108.193, 103.102, 108.451, 108.342)

In [59]:
Tm=FcM(fcs=[1,0,0,0,-1],atom=ch4str,unit="bohr",basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
Tmf = scf.RHF(Tm)
Tmf.scf(dm0=Tmf.init_guess_by_1e())

converged SCF energy = -56.2053208898247


-56.20532088982467

In [60]:
Tgeom_eq=optimize(Tmf)


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   C   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   H   0.000000   0.000000   1.081561    0.000000  0.000000  0.000000
   H   0.000000   1.019706  -0.360520    0.000000  0.000000  0.000000
   H   0.883091  -0.509853  -0.360520    0.000000  0.000000  0.000000
   H  -0.883091  -0.509853  -0.360520    0.000000  0.000000  0.000000
converged SCF energy = -56.2053208898294
--------------- SCF_Scanner gradients ---------------
         x                y                z
0 C    -0.0379250204    -0.0218960208    -0.0154828248
1 H    -0.0052085176    -0.0030071391     0.0634324802
2 H    -0.0052085176     0.0588023362    -0.0239793180
3 H     0.0483200581    -0.0339118767    -0.0239793180
4 H     0.0000219976     0.0000127003     0.0000089805
----------------------------------------------
cycle 1: E = -56.2053208898  dE = -56.2053  norm(grad) = 0.1

/home/administrator/anaconda3/lib/python3.7/site-packages/berny/Math.py:19: RuntimeWarning: divide by zero encountered in true_divide
  gaps = D[:-1] / D[1:]


In [61]:
Tgeom_eqcrds=Tgeom_eq.atom_coords()
Tgeom_eqcrds-=Tgeom_eqcrds[0]
norm(Tgeom_eqcrds[1]),anglecenter(Tgeom_eqcrds)

(1.8863278671756294, 108.172)

In [62]:
Tg=Tmf.Gradients().grad()
Th=Tmf.Hessian().hess()

--------------- RHF gradients ---------------
         x                y                z
0 C    -0.0379252172    -0.0218961344    -0.0154829051
1 H    -0.0052084464    -0.0030070979     0.0634324768
2 H    -0.0052084464     0.0588023467    -0.0239792781
3 H     0.0483201028    -0.0339118203    -0.0239792781
4 H     0.0000220072     0.0000127059     0.0000089844
----------------------------------------------


In [63]:
Tcoords=Tm.atom_coords()
Targg=(ch4coords-np.einsum('ijkl,kl->ij',invhess(Th), Tg ))[:4]

In [64]:
Targg-=Targg[0]

In [65]:
norm(Targg[1]),anglecenter(Targg)

(1.8305452430146139, 109.12)

In [66]:
af_1-af_1[0],g_nh3_2-g_nh3_2[0]

(array([[ 0.     ,  0.     ,  0.     ],
        [ 0.01291,  0.00746,  0.0783 ],
        [ 0.01291,  0.0763 , -0.01907],
        [ 0.07254, -0.02697, -0.01907],
        [ 0.02872,  0.01658,  0.01172]]), array([[0.     , 0.     , 0.     ],
        [0.11377, 0.06569, 0.11445],
        [0.11377, 0.1298 , 0.02378],
        [0.1693 , 0.03363, 0.02378],
        [0.19945, 0.11515, 0.08142]]))

In [67]:
Tg

array([[-0.03793, -0.0219 , -0.01548],
       [-0.00521, -0.00301,  0.06343],
       [-0.00521,  0.0588 , -0.02398],
       [ 0.04832, -0.03391, -0.02398],
       [ 0.00002,  0.00001,  0.00001]])

In [68]:
af_1,g_nh3_2,g_nh3_3

(array([[-0.02542, -0.01467, -0.01038],
        [-0.0125 , -0.00722,  0.06792],
        [-0.0125 ,  0.06163, -0.02945],
        [ 0.04712, -0.04164, -0.02945],
        [ 0.0033 ,  0.00191,  0.00135]]),
 array([[-0.11926, -0.06885, -0.04869],
        [-0.00549, -0.00317,  0.06576],
        [-0.00549,  0.06095, -0.02491],
        [ 0.05004, -0.03522, -0.02491],
        [ 0.08019,  0.0463 ,  0.03274]]),
 array([[-0.06037, -0.03485, -0.02465],
        [-0.004  , -0.00231,  0.0639 ],
        [-0.004  ,  0.05947, -0.02347],
        [ 0.04951, -0.0332 , -0.02347],
        [ 0.01886,  0.01089,  0.0077 ]]))

Look at https://en.wikipedia.org/wiki/Bond-dissociation_energy for approximate BDE 

In [69]:
def mpa(g,h,R0,De):
    Z=g**2/(2*De)/h
    print(np.roots([1,-2,1-2*Z,+Z]))
    t,a=0,0
    for i in range(2):
        t=float(np.roots([1,-2,1-2*Z,+Z])[i])
        a=g/(2*De)/(t*(1-t))
        if a>0 and t>0:
            break
    dX=np.log(t)/a
    print(t,a)
    return R0+dX

In [70]:
mpa(Tg[1,2],h0[1,1,2,2],norm(ch4coords[1]),.2)

[ 1.17715  0.85011 -0.02726]
0.8501073052675587 1.2445073406328915


1.9133667057308597

In [71]:
norm(ch4coords[1])-Tg[1,2]/Th[1,1,2,2]

1.8312577565319843

In [72]:
norm(ch4coords[1])

2.043854241378284

In [73]:
from berny import Berny, geomlib, coords,Geometry

In [74]:
pbg=Geometry(['N','H','H','H'],ch4coords[:4]*bohr2ang)

In [75]:
b=Berny(pbg,debug=True)
b.send((Tmf.e_tot,Tg[:4]))['predicted']

Point(q=array([1.91607, 1.91607, 1.91607, 1.84605, 1.84605, 1.84605]), E=-56.219490717390556, g=None)

In [76]:
1.84605*180/np.pi

105.77087377012562

In [77]:
b=Berny(pbg,debug=True)
b.send((Tmf.e_tot,af_1[:4]))['predicted']

Point(q=array([1.9173 , 1.9173 , 1.9173 , 1.79238, 1.79238, 1.79238]), E=-56.226300014472, g=None)

In [78]:
1.79238*180/np.pi

102.69580928365849

In [79]:
b=Berny(pbg,debug=True)
b.send((Tmf.e_tot,g_nh3_3[:4]))['predicted']

Point(q=array([1.91114, 1.91114, 1.91114, 1.91955, 1.91955, 1.91955]), E=-56.219143201658994, g=None)

In [80]:
1.91955*180/np.pi

109.98211356433717

In [81]:
pbg2=Geometry(['O','H','H'],ch4coords[:3]*bohr2ang)

In [82]:
b2=Berny(pbg2,debug=True)
q=b2.send((Tmf.e_tot,(2*afC+afH1+afH2)[:3]))['predicted'].q
q[0],q[2]*180/np.pi

(1.88342732347918, 98.22512188533129)

In [83]:
pbg3=Geometry(['F','H'],ch4coords[:2]*bohr2ang)
b3=Berny(pbg3,debug=True)
b3.send((Tmf.e_tot,(3*afC+afH1+afH2+afH3)[:2]))['predicted']

Point(q=array([1.80025]), E=-56.22047454179093, g=None)